<!-- ## SMALL-SCALE CROP-MAPPING(Food security) -->

<center><h1>NDVI Application</h1> </center>

In [1]:
#Importation of variAous Libraries or packages...
import os
from os import path as op
import geopandas as gdp
import geemap
import ee
import geemap.foliumap as geemap

# Making module or packages collection attribute became callable..
import collections
collections.Callable = collections.abc.Callable

#plotting packages
import pandas as pd#Used for data analysis also include DataFrame df data structure
import matplotlib.pyplot as plt#For plotting of dataframes
import numpy as np
# import altair as alt#declarative visualization(Used for charting)

import ipywidgets as widgets

from ipyleaflet import WidgetControl
from ipywidgets import HTML
from IPython.display import display, clear_output
from ipywidgets import AppLayout

from geemap import chart #for chart plotting

import datetime
from pylab import *
import seaborn as sns
from matplotlib.pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose


#Main function is to out various warning that might arise such as new upadates in package version
import warnings
warnings.filterwarnings("ignore")

import folium

Map=geemap.Map()

from geemap import Map
from ipywidgets import Button, Layout

map_layout= Map(layout=Layout(height='100px'))
map_layout.setCenter(-121.842, 38.855,6)
 
import geemap.chart as chart


In [2]:
#Setting various buttons for the final dsplay of the results and the output widgets or setting the intractive keys
style = {'description_width': 'initial'}
#General output widget
output_widget = widgets.Output(layout={'border': '1px solid red','align-items':'center','background-color':'red'})


#General output control tool.
output_control = WidgetControl(widget=output_widget, position='topright')
#Map output control
map_layout.add_control(output_control)

#BUTTON  and TEXT SECTION 
Enter_project = widgets.Text(
    placeholder='Enter a project name(if any)',
    description='project selection:',
    disabled=False,
)

#styling method that can be used-----'primary', 'success', 'info', 'warning', 'danger',)
#Data collection button: Selection of collected data from the field.

#AOI Area of Interest Selection of study area.
AOI = widgets.Button(
    description='Drawn ROI',
    button_style='success',
    tooltip='Draw you boundaries',
    style=style
)

Load_Data = widgets.Button(
    description='Load Data',
    button_style='info',
    tooltip='Load NDVI Data',
    style=style
)
Cal_Ndvi= widgets.Button(
    description='Extract NDVI',
    button_style='warning',
    tooltip='Analysis Charts',
    style=style
)
Plot_Chart= widgets.Button(
    description='Analysis Chart',
    button_style='primary',
    tooltip='Timeseries charts',
    style=style
)

reset = widgets.Button(
    description='Reset',
    button_style='danger',
    tooltip='Reset me',
    style=style
)

In [3]:
#Arrange the layout
from ipywidgets import AppLayout
verticalBox = output_widget
vBox1 = widgets.VBox([map_layout])


vBox2 = widgets.VBox([AOI,Load_Data,Cal_Ndvi,Plot_Chart,reset])

AppLayout(header=None,
          left_sidebar=vBox1,
          right_sidebar=vBox2,
          footer=None,
          step= 1,
          pane_widths=[3, 7, 1],
          pane_heights=[1, 1, '100%'])

AppLayout(children=(VBox(children=(Map(center=[38.855, -121.842], controls=(WidgetControl(options=['position',…

In [4]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
            try:
                global boundaries
                boundaries=map_layout.user_rois.geometry()
                map_layout.addLayer(boundaries,{},"Region of Intrest")
            except Exception as e:
                print('Please select Your area of Interest.....')
            else:
                print("successfully drawn your Region of Interest ")
AOI.on_click(AOI_clicked)

In [5]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
            try:
                # Create image collection
                # Set start and end date
                startTime = datetime.datetime(2015, 1, 1)
                endTime = datetime.datetime(2020, 12, 31)
                global collection
                collection = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M').filterDate(startTime, endTime)
                colorizedVis = {
                  'min': 0.0,
                  'max': 1.0,
                  'palette': [
                    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                    '012E01', '011D01', '011301'
                  ],
                }
                 
                colorizedVis = {'min':0, 'max':200, 'palette':['red','brown','yellow', 'green'] }#NDVI visualization parameters
                NDVI=collection.select("NDVI")
                global collection_clip
                collection_clip=NDVI.mean().clip(boundaries)
                map_layout.centerObject(boundaries,12)
                map_layout.addLayer(collection_clip, colorizedVis, 'Colorized')

            except Exception as e:
                print('Please select repeat the previous steps.....')
            else:
                print("successfully clipped the data to a specific region within the map ")
        
Load_Data.on_click(AOI_clicked)

In [6]:

#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
            try:
                info = collection.getRegion(boundaries,5000).getInfo()
                # Reshape image collection 
                header = info[0]
                data = array(info[1:])

                iTime = header.index('time')
                time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(float))]

                # List of used image bands
                band_list = ['RED',u'NIR']

                iBands = [header.index(b) for b in band_list]
                yData = data[0:,iBands].astype(np.float)

                # Calculate NDVI
                red = yData[:,0]
                nir = yData[:,1]
                global ndvi
                ndvi = (nir - red) / (nir + red)

            except Exception as e:
                print('Please select repeat the previous steps.....')
            else:
                print("successfully estimated the NDVI")       
Cal_Ndvi.on_click(AOI_clicked)

In [7]:
user_out = widgets.Output(layout={'border': '1px solid green'})

In [8]:

#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with user_out:
            output_widget.clear_output()
            try:
                df = pd.DataFrame(data=ndvi, index=list(range(len(ndvi))), columns=['NDVI'])
                df = df.interpolate()
                df['Date'] = pd.Series(time, index=df.index)
                df = df.set_index(df.Date)
                df.index = pd.to_datetime(df.index)
                df['NDVI']=df['NDVI'].fillna(0)

                # Change size
                sns.set(rc={'figure.figsize':(15, 6)})

                df_monthly=df.resample('W').mean()
                df_monthly['pct_change'] = df_monthly.pct_change(52)
                df_monthly['pct_change']['2016':].plot()
                plt.title('NDVI Weekly Percent Change')
                
            except Exception as e:
                print('Please select repeat the previous steps.....')
            else:
                print("successfully estimated the NDVI")
        
Plot_Chart.on_click(AOI_clicked)

In [9]:
user_out

Output(layout=Layout(border='1px solid green'))

<!-- <center><img src="mylogo.png" style="width: 80px;border-radius: 60%;
"></center> -->